<a href="https://colab.research.google.com/github/Nisar8856/Simplified_Binance_Futures_Trading_Bot.ipynb/blob/main/Simplified_Binance_Futures_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.6 MB/s eta 0:00:00


In [ ]:
import logging
import sys
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException

# --- Configuration ---
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("trading_bot.log"), # Log to a file
        logging.StreamHandler(sys.stdout)       # Log to console
    ]
)
logger = logging.getLogger('TradingBot')

class BasicBot:
    """
    A simplified trading bot for Binance Futures Testnet.
    Supports placing MARKET, LIMIT, and STOP_MARKET orders.
    """
    def __init__(self, api_key: str, api_secret: str, testnet: bool = True):
        """
        Initializes the Binance client.

        Args:
            api_key (str): Your Binance API key.
            api_secret (str): Your Binance API secret.
            testnet (bool): True to connect to Binance Testnet, False for production.
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.testnet = testnet

        if self.testnet:
            # For Binance Futures Testnet, setting testnet=True is sufficient.
            # The base_url is automatically handled by the client for Futures methods.
            self.client = Client(api_key, api_secret, testnet=True)
            logger.info("Connected to Binance Futures Testnet.")
        else:
            # For Binance Futures Production (use with extreme caution!)
            self.client = Client(api_key, api_secret)
            logger.warning("Connected to Binance Futures Production. Use with extreme caution!")

        try:
            # Verify connectivity
            server_time = self.client.get_server_time()
            logger.info(f"Successfully connected to Binance. Server time: {server_time['serverTime']}")
        except (BinanceAPIException, BinanceRequestException) as e:
            logger.error(f"Failed to connect to Binance API: {e}")
            sys.exit(1) # Exit if connection fails

    def _log_api_call(self, endpoint: str, params: dict, response: dict = None, error: Exception = None):
        """
        Logs details of an API request and its response/error.

        Args:
            endpoint (str): The API endpoint being called (e.g., 'futures_create_order').
            params (dict): Parameters sent with the request.
            response (dict, optional): The API response. Defaults to None.
            error (Exception, optional): Any exception encountered. Defaults to None.
        """
        logger.info(f"API Call: {endpoint}, Params: {params}")
        if response:
            logger.info(f"API Response: {response}")
        if error:
            logger.error(f"API Error: {error}")

    def get_account_balance(self, asset: str = 'USDT') -> float:
        """
        Retrieves the balance for a specific asset in your Futures account.

        Args:
            asset (str): The asset to check balance for (e.g., 'USDT').

        Returns:
            float: The available balance of the asset, or 0.0 if not found/error.
        """
        try:
            account_info = self.client.futures_account_balance()
            self._log_api_call('futures_account_balance', {}, account_info)
            for balance in account_info:
                if balance['asset'] == asset:
                    # 'availableBalance' is the balance that can be used for trading
                    logger.info(f"Available {asset} Balance: {balance['availableBalance']}")
                    return float(balance['availableBalance'])
            logger.warning(f"{asset} balance not found in account info.")
            return 0.0
        except (BinanceAPIException, BinanceRequestException) as e:
            self._log_api_call('futures_account_balance', {}, error=e)
            logger.error(f"Error fetching account balance: {e}")
            return 0.0

    def get_symbol_info(self, symbol: str) -> dict:
        """
        Retrieves exchange information for a given symbol, including filters for price and quantity.

        Args:
            symbol (str): The trading pair symbol (e.g., 'BTCUSDT').

        Returns:
            dict: A dictionary containing symbol information, or an empty dict on error.
        """
        try:
            exchange_info = self.client.futures_exchange_info()
            for s in exchange_info['symbols']:
                if s['symbol'] == symbol.upper():
                    info = {
                        'status': s['status'],
                        'price_precision': s['pricePrecision'],
                        'quantity_precision': s['quantityPrecision'],
                        'min_notional': 0.0, # Default
                        'min_qty': 0.0,      # Default
                        'tick_size': 0.0     # Default
                    }
                    for f in s['filters']:
                        if f['filterType'] == 'MIN_NOTIONAL':
                            info['min_notional'] = float(f['notional'])
                        elif f['filterType'] == 'MARKET_LOT_SIZE':
                            info['min_qty'] = float(f['minQty'])
                        elif f['filterType'] == 'PRICE_FILTER':
                            info['tick_size'] = float(f['tickSize'])
                    logger.info(f"Symbol info for {symbol}: {info}")
                    return info
            logger.warning(f"Symbol {symbol} not found in exchange information.")
            return {}
        except (BinanceAPIException, BinanceRequestException) as e:
            self._log_api_call('futures_exchange_info', {'symbol': symbol}, error=e)
            logger.error(f"Error fetching symbol info for {symbol}: {e}")
            return {}

    def _validate_order_params(self, symbol: str, quantity: float, price: float = None, stop_price: float = None) -> bool:
        """
        Validates order parameters against Binance exchange rules.

        Args:
            symbol (str): The trading pair symbol.
            quantity (float): The order quantity.
            price (float, optional): The price for limit orders. Defaults to None.
            stop_price (float, optional): The stop price for stop orders. Defaults to None.

        Returns:
            bool: True if parameters are valid, False otherwise.
        """
        if quantity <= 0:
            logger.error("Quantity must be greater than 0.")
            return False
        if price is not None and price <= 0:
            logger.error("Price must be greater than 0.")
            return False
        if stop_price is not None and stop_price <= 0:
            logger.error("Stop price must be greater than 0.")
            return False

        symbol_info = self.get_symbol_info(symbol)
        if not symbol_info:
            logger.error(f"Could not retrieve symbol information for {symbol}.")
            return False

        if symbol_info.get('status') != 'TRADING':
            logger.error(f"Symbol {symbol} is not currently trading (status: {symbol_info.get('status')}).")
            return False

        # Validate quantity precision and minimum quantity
        min_qty = symbol_info.get('min_qty', 0.0)
        quantity_precision = symbol_info.get('quantity_precision', 0)

        if quantity < min_qty:
            logger.error(f"Quantity {quantity} is less than minimum allowed quantity {min_qty} for {symbol}.")
            return False

        # Check quantity precision
        if quantity_precision > 0:
            # Convert quantity to string to check decimal places
            qty_str = str(quantity)
            if '.' in qty_str:
                decimal_places = len(qty_str.split('.')[1])
                if decimal_places > quantity_precision:
                    logger.error(f"Quantity {quantity} exceeds precision of {quantity_precision} decimal places for {symbol}.")
                    return False

        # Validate price precision if price is provided
        if price is not None:
            price_precision = symbol_info.get('price_precision', 0)
            if price_precision > 0:
                price_str = str(price)
                if '.' in price_str:
                    decimal_places = len(price_str.split('.')[1])
                    if decimal_places > price_precision:
                        logger.error(f"Price {price} exceeds precision of {price_precision} decimal places for {symbol}.")
                        return False
            # Check price against tick size (optional but good practice)
            tick_size = symbol_info.get('tick_size', 0.0)
            if tick_size > 0 and (price % tick_size != 0):
                # This check can be tricky with floating point, better to round price to precision
                pass # We rely more on precision rounding

        if stop_price is not None:
            price_precision = symbol_info.get('price_precision', 0)
            if price_precision > 0:
                stop_price_str = str(stop_price)
                if '.' in stop_price_str:
                    decimal_places = len(stop_price_str.split('.')[1])
                    if decimal_places > price_precision:
                        logger.error(f"Stop price {stop_price} exceeds precision of {price_precision} decimal places for {symbol}.")
                        return False

        return True

    def place_order(self, symbol: str, side: str, order_type: str, quantity: float, price: float = None, stop_price: float = None):
        """
        Places an order on Binance Futures.

        Args:
            symbol (str): The trading pair (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            order_type (str): 'MARKET', 'LIMIT', or 'STOP_MARKET'.
            quantity (float): The amount to trade.
            price (float, optional): The price for LIMIT orders. Required for LIMIT.
            stop_price (float, optional): The stop price for STOP_MARKET orders. Required for STOP_MARKET.
        """
        symbol = symbol.upper()
        side = side.upper()
        order_type = order_type.upper()

        if side not in ['BUY', 'SELL']:
            logger.error("Invalid side. Must be 'BUY' or 'SELL'.")
            return

        # Validate parameters before attempting API call
        if not self._validate_order_params(symbol, quantity, price, stop_price):
            logger.error("Order parameters failed validation. Order not placed.")
            return

        order_params = {
            'symbol': symbol,
            'side': side,
            'type': order_type,
            'quantity': f'{quantity:.{self.get_symbol_info(symbol).get("quantity_precision", 0)}f}' # Format quantity to precision
        }

        if order_type == 'LIMIT':
            if price is None:
                logger.error("Price is required for LIMIT orders.")
                return
            order_params['price'] = f'{price:.{self.get_symbol_info(symbol).get("price_precision", 0)}f}' # Format price to precision
            order_params['timeInForce'] = 'GTC' # Good Till Cancelled
        elif order_type == 'STOP_MARKET':
            if stop_price is None:
                logger.error("Stop price is required for STOP_MARKET orders.")
                return
            order_params['stopPrice'] = f'{stop_price:.{self.get_symbol_info(symbol).get("price_precision", 0)}f}' # Format stop price to precision
            # For STOP_MARKET, the order is placed as a MARKET order once stopPrice is reached.
            # No 'price' is needed for the order itself.
        elif order_type == 'MARKET':
            # No price or stopPrice needed for MARKET orders
            pass
        else:
            logger.error(f"Unsupported order type: {order_type}. Supported: MARKET, LIMIT, STOP_MARKET.")
            return

        try:
            logger.info(f"Attempting to place {order_type} {side} order for {quantity} {symbol}...")
            response = self.client.futures_create_order(**order_params)
            self._log_api_call('futures_create_order', order_params, response)
            logger.info(f"Order placed successfully!")
            logger.info(f"Order ID: {response.get('orderId')}")
            logger.info(f"Status: {response.get('status')}")
            logger.info(f"Type: {response.get('type')}")
            logger.info(f"Side: {response.get('side')}")
            logger.info(f"Symbol: {response.get('symbol')}")
            logger.info(f"Quantity: {response.get('origQty')}")
            if response.get('price'):
                logger.info(f"Price: {response.get('price')}")
            if response.get('stopPrice'):
                logger.info(f"Stop Price: {response.get('stopPrice')}")
            return response
        except BinanceAPIException as e:
            self._log_api_call('futures_create_order', order_params, error=e)
            logger.error(f"Binance API Error placing order: {e.code} - {e.message}")
        except BinanceRequestException as e:
            self._log_api_call('futures_create_order', order_params, error=e)
            logger.error(f"Binance Request Error placing order: {e}")
        except Exception as e:
            self._log_api_call('futures_create_order', order_params, error=e)
            logger.error(f"An unexpected error occurred: {e}")
        return None

def main():
    """
    Main function to run the command-line interface for the trading bot.
    """
    print("\n--- Simplified Binance Futures Trading Bot ---")
    print("Please ensure you have generated API credentials for Binance Testnet.")
    print("WARNING: Do NOT use your production API keys here.")

    api_key = input("Enter your Binance Testnet API Key: ").strip()
    api_secret = input("Enter your Binance Testnet API Secret: ").strip()

    if not api_key or not api_secret:
        logger.error("API Key and Secret cannot be empty. Exiting.")
        sys.exit(1)

    bot = BasicBot(api_key, api_secret, testnet=True)

    while True:
        print("\n--- Main Menu ---")
        print("1. Place Order")
        print("2. Check Account Balance (USDT)")
        print("3. Exit")
        choice = input("Enter your choice (1-3): ").strip()

        if choice == '1':
            print("\n--- Place Order ---")
            symbol = input("Enter symbol (e.g., BTCUSDT): ").strip().upper()
            side = input("Enter side (BUY/SELL): ").strip().upper()
            order_type = input("Enter order type (MARKET/LIMIT/STOP_MARKET): ").strip().upper()

            try:
                quantity = float(input("Enter quantity: ").strip())
            except ValueError:
                logger.error("Invalid quantity. Please enter a number.")
                continue

            price = None
            stop_price = None

            if order_type == 'LIMIT':
                try:
                    price = float(input("Enter limit price: ").strip())
                except ValueError:
                    logger.error("Invalid price. Please enter a number.")
                    continue
            elif order_type == 'STOP_MARKET':
                try:
                    stop_price = float(input("Enter stop price: ").strip())
                except ValueError:
                    logger.error("Invalid stop price. Please enter a number.")
                    continue
            elif order_type == 'MARKET':
                pass # No additional price input needed
            else:
                logger.error("Invalid order type. Please choose MARKET, LIMIT, or STOP_MARKET.")
                continue

            bot.place_order(symbol, side, order_type, quantity, price, stop_price)

        elif choice == '2':
            bot.get_account_balance('USDT')

        elif choice == '3':
            logger.info("Exiting bot. Goodbye!")
            break
        else:
            logger.warning("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()


--- Simplified Binance Futures Trading Bot ---
Please ensure you have generated API credentials for Binance Testnet.

--- Main Menu ---
1. Place Order
2. Check Account Balance (USDT)
3. Exit

--- Place Order ---


ERROR:TradingBot:Invalid order type. Please choose MARKET, LIMIT, or STOP_MARKET.



--- Main Menu ---
1. Place Order
2. Check Account Balance (USDT)
3. Exit

--- Place Order ---


ERROR:TradingBot:Invalid quantity. Please enter a number.



--- Main Menu ---
1. Place Order
2. Check Account Balance (USDT)
3. Exit

--- Place Order ---


ERROR:TradingBot:Invalid quantity. Please enter a number.



--- Main Menu ---
1. Place Order
2. Check Account Balance (USDT)
3. Exit

--- Place Order ---


ERROR:TradingBot:Invalid order type. Please choose MARKET, LIMIT, or STOP_MARKET.



--- Main Menu ---
1. Place Order
2. Check Account Balance (USDT)
3. Exit
